<a href="https://colab.research.google.com/github/FaiazS/AI-Powered-E-Commerce-Product-Advisor/blob/main/Product_Recommendation_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain

!pip install langchain-groq

!pip install langchain-community

!pip install rich

!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [2]:
import os

from google.colab import userdata

from langchain_groq import ChatGroq

groq_api_key = os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

llama = ChatGroq(

                 model = 'llama-3.3-70b-versatile'
)

In [3]:
import pandas as pd

products_dataframe = pd.read_csv('/content/ecommerce_products_dataset.csv')

products_dataframe.head()

,Product ID,Product Name,Category,Price_USD,Description,Rating
0,P00001,Science Fiction Novel,Books,103.82,Live while professor white we writer determine...,4.4
1,P00002,Smartwatch,Electronics,904.61,Heart rate monitor; GPS tracking; AMOLED displ...,4.6
2,P00003,Laptop,Electronics,621.59,Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-...,4.1
3,P00004,Sneakers,Fashion,164.85,Others feeling billion lawyer evening enter eye.,3.0
4,P00005,Laptop,Electronics,408.08,Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-...,4.8


In [4]:
products_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product ID    100 non-null    object 
 1   Product Name  100 non-null    object 
 2   Category      100 non-null    object 
 3   Price_USD     100 non-null    float64
 4   Description   100 non-null    object 
 5   Rating        100 non-null    float64
dtypes: float64(2), object(4)
memory usage: 4.8+ KB


In [5]:
products_dataframe.describe()

,Price_USD,Rating
count,100.000000,100.000000
mean,551.212900,4.097000
std,448.910608,0.568261
min,10.430000,3.000000
25%,145.307500,3.600000
50%,407.175000,4.200000
75%,914.097500,4.600000
max,1467.960000,5.000000


In [6]:
products_dataframe['Category'].unique()

array(['Books', 'Electronics', 'Fashion', 'Home & Kitchen',
       'Toys & Games'], dtype=object)

In [7]:
products_dataframe['Rating'].unique()

array([4.4, 4.6, 4.1, 3. , 4.8, 3.1, 3.9, 3.6, 3.7, 3.4, 3.8, 4.3, 4. ,
       4.7, 4.5, 3.2, 4.9, 4.2, 5. , 3.3, 3.5])

**Pandas Code Tool Executor Chain**

In [8]:
from langchain_core.runnables import chain

from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

from langchain_core.prompts import ChatPromptTemplate


@chain
def pandas_code_tool_executor(pandas_query):

  products_dataframe_filtered = eval(pandas_query)

  if products_dataframe_filtered.empty:

    return products_dataframe.to_markdown()

  else:

    return products_dataframe_filtered.to_markdown()


product_filtering_prompt = """

Given the following schema of a dataframe table,
            your task is to figure out the best pandas query to
            filter the dataframe based on the user query which
            will be in natural language.

            The schema is as follows:

            #   Column        Non-Null Count  Dtype
            ---  ------        --------------  -----
            0   Product ID    30 non-null     object
            1   Product Name  30 non-null     object
            2   Category      30 non-null     object
            3   Price_USD     30 non-null     int64
            4   Description   30 non-null     object
            5   Rating        30 non-null     float64

            Category has values: ['Books','Electronics','Fashion',
                                  'Toys & Games','Home & Kitchen']

            Rating ranges from 1 - 5 in floats

            You will try to figure out the pandas query focusing
            only on Category, Price_USD and Rating if the user mentions
            anything about these in their natural language query.
            Do not make up column names, only use the above.
            If not the pandas query should just return the full dataframe.
            Remember the dataframe name is products_dataframe.

            Just return only the pandas query and nothing else.
            Do not return the results as markdown, just return the query

            User Query: {user_query}

            Pandas Query:
        """

prompt_template_1 = ChatPromptTemplate.from_template(product_filtering_prompt)

product_filtering_chain = prompt_template_1 | llama | StrOutputParser() | pandas_code_tool_executor

product_filtering_result = product_filtering_chain.invoke({'user_query': 'looking for a product in the Electronics category with ratings above 4 and priced below 200'})

print(product_filtering_result)

|    | Product ID   | Product Name      | Category    |   Price_USD | Description                                                                  |   Rating |
|---:|:-------------|:------------------|:------------|------------:|:-----------------------------------------------------------------------------|---------:|
| 33 | P00034       | Bluetooth Speaker | Electronics |       58.22 | 10W output; 12-hour battery; Water-resistant; Bluetooth 5.0                  |      4.7 |
| 39 | P00040       | Laptop            | Electronics |      165.93 | Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD screen                |      4.2 |
| 45 | P00046       | Bluetooth Speaker | Electronics |      105.5  | 10W output; 12-hour battery; Water-resistant; Bluetooth 5.0                  |      4.9 |
| 69 | P00070       | Wireless Earbuds  | Electronics |       64.2  | Noise cancellation; Touch controls; 24-hour battery with case; Bluetooth 5.2 |      4.5 |
| 78 | P00079       | Laptop            

In [9]:
product_recommendation_prompt = """

                       Act as an expert retail product advisor.

                       Given the following table of products, focus on the product attributes and description in the table and based on the user query below do the following:

                         - Recommend the most appropriate products based on the user query.

                         - Recommedation should have product name, price, rating, description.

                         - Also add a brief on why you recommend the product.

                         - Do not make up products or recommend products that are not present in the table.

                         - If some specifications do not match focus on the ones which match and recommend.

                         - If nothing matches recommend 5 random products from the table.

                         - Do not generate anything else except the fields mentioned above.

Just in case if the user query is a generic query or greeting respond to them appropriately without recommending any products.

Product Table:

{product_filtering_result}

User Query:

{user_query}

Recommendation:

"""

prompt_template_2 = ChatPromptTemplate.from_template(product_recommendation_prompt)

product_recommendation_chain = prompt_template_2 | llama | StrOutputParser()

product_recommendation_result = product_recommendation_chain.invoke({'product_filtering_result': product_filtering_result, 'user_query': 'Looking for a laptop with more than 8GB RAM'})

print(product_recommendation_result)

Product Name: Laptop
Price: $165.93
Rating: 4.2
Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD screen
Brief: I recommend this product because it meets the user's requirement of having more than 8GB RAM, specifically 16GB RAM, making it a suitable option for their needs.

Product Name: Laptop
Price: $50.25
Rating: 4.1
Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD screen
Brief: I recommend this product because it also meets the user's requirement of having more than 8GB RAM, with 16GB RAM, and is another option to consider.


In [10]:
product_browse_chain = {'user_query': itemgetter('user_query'), 'product_filtering_result': product_filtering_chain} | product_recommendation_chain

product_browsing_result = product_browse_chain.invoke({'user_query': 'Looking for earbuds with a long lasting battery life and minimal charging needs!'})

print(product_browsing_result)

Product Name: Wireless Earbuds
Price: $64.2
Rating: 4.5
Description: Noise cancellation; Touch controls; 24-hour battery with case; Bluetooth 5.2
Why: I recommend this product because it has a 24-hour battery life with the case, which matches your requirement of long lasting battery life and minimal charging needs.

Product Name: Wireless Earbuds
Price: $244.95
Rating: 4.2
Description: Noise cancellation; Touch controls; 24-hour battery with case; Bluetooth 5.2
Why: I recommend this product because it also has a 24-hour battery life with the case, which aligns with your need for long lasting battery life.

Product Name: Wireless Earbuds
Price: $834.09
Rating: 4.9
Description: Noise cancellation; Touch controls; 24-hour battery with case; Bluetooth 5.2
Why: I recommend this product because it has a 24-hour battery life with the case and a high rating, making it a reliable choice for long lasting battery life.

Product Name: Wireless Earbuds
Price: $322.49
Rating: 4.6
Description: Noise 

**Historical Conversation Query Rephraser Chain**

In [11]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

from langchain_core.chat_history import BaseChatMessageHistory

from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.runnables import RunnablePassthrough

from langchain_core.output_parsers import StrOutputParser


def get_session_history(session_id):

   db_url = 'sqlite:///conversation_history.db'

   if db_url is not None:

    print(f"Established SQLite Database connection: {db_url}")

    return SQLChatMessageHistory(session_id=session_id, connection_string=db_url)

In [16]:
system_prompt = """

You are an ecommerce product expert. Your task is to carefully analyze the given conversation history

and the current user query and rephrase the current user query to a standalone query which can be used without the history for making search queries.

However rephrase only if needed. Strictly and only and only return the query and do not answer it.

"""

current_user_query = """

Current User Query:

{current_user_query}

"""

product_conversation_history = ChatPromptTemplate.from_messages(

                                                  [

                                                      ('system', system_prompt),

                                                      MessagesPlaceholder(variable_name='history'),

                                                      ('user', current_user_query)
                                                   ]

     )

In [17]:
#Memory buffer window function to return the last K conversations

def memory_buffer_window(messages, k = 10):

  return messages[-(k*2):]

In [31]:
rephrased_user_query_chain = RunnableWithMessageHistory(

                                                        product_conversation_history,

                                                        get_session_history,

                                                        input_messages_key= 'current_user_query',

                                                        history_messages_key= 'history'
)

In [38]:
final_product_browsing_chain_flow = {'current_user_query': itemgetter('current_user_query')} | RunnablePassthrough.assign(user_query=rephrased_user_query_chain) | RunnablePassthrough.assign(product_filtering_result=product_filtering_chain) | product_recommendation_chain

**Final Wrap Up: Integrating Memory**

In [39]:
from rich.console import Console

from rich.markdown import Markdown

final_product_shopping_flow = RunnableWithMessageHistory(

                                                         final_product_browsing_chain_flow,

                                                         get_session_history,

                                                         input_messages_key= 'current_user_query',

                                                         history_messages_key= 'history'
)

def chat_with_e_commerce_agent(prompt: str, session_id: str):

  response = final_product_shopping_flow.invoke({'current_user_query': prompt},  {'configurable': {'session_id' : session_id}})

  console = Console()

  console.print(Markdown(response))

In [40]:
user_id = '0001'

customer_prompt = 'I am looking for a laptop'

chat_with_e_commerce_agent(customer_prompt, user_id)

Established SQLite Database connection: sqlite:///conversation_history.db
Established SQLite Database connection: sqlite:///conversation_history.db


Product Name: Laptop Price: 408.08 Rating: 4.8 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it matches the user's query for a laptop and has a high rating of 4.8,     
indicating good performance and customer satisfaction.                                                             

Product Name: Laptop Price: 621.59 Rating: 4.1 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it is another laptop option with similar specifications, although with a   
lower rating of 4.1.                                                                                               

Product Name: Laptop Price: 710.99 Rating: 3.1 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it is a laptop option, but with a lower rating of 3.1, it may not be as    
highly rated as other options.                                                                                     

Product Name: Laptop Price: 553.46 Rating: 3.6 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it is another laptop option with a rating of 3.6, which is relatively      
average.                                                                                                           

Product Name: Laptop Price: 1015.98 Rating: 3.8 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD
screen I recommend this product because it is a higher-priced laptop option with a rating of 3.8, which may        
indicate better performance and features.

In [41]:
user_id = '0001'

customer_prompt = "I want in the range 150 - 300"

chat_with_e_commerce_agent(customer_prompt, user_id)

Established SQLite Database connection: sqlite:///conversation_history.db
Established SQLite Database connection: sqlite:///conversation_history.db


Product Name: Laptop Price: 165.93 Rating: 4.2 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it matches the user's query for a laptop and falls within the specified    
price range of 150-300.                                                                                            

Since there is only one laptop in the given price range, I will recommend four more random products from the table:

Product Name: Smartphone Price: 190.5 Rating: 3.9 Description: 6.5-inch OLED display; 128GB storage; Triple-lens   
camera; 5G connectivity I recommend this product because it is a high-end smartphone with advanced features.       

Product Name: Smartphone Price: 165.03 Rating: 3.8 Description: 6.5-inch OLED display; 128GB storage; Triple-lens  
camera; 5G connectivity I recommend this product because it is another smartphone option with similar              
specifications.                                                                                                    

Product Name: Smartwatch Price: 213.87 Rating: 4.8 Description: Heart rate monitor; GPS tracking; AMOLED display;  
7-day battery life I recommend this product because it is a highly-rated smartwatch with advanced health and       
fitness features.                                                                                                  

Product Name: Wireless Earbuds Price: 244.95 Rating: 4.2 Description: Noise cancellation; Touch controls; 24-hour  
battery with case; Bluetooth 5.2 I recommend this product because it is a high-quality audio device with advanced  
features.

In [42]:
user_id = '0001'

customer_prompt = 'What is the color of this laptop by the way?'

chat_with_e_commerce_agent(customer_prompt, user_id)

Established SQLite Database connection: sqlite:///conversation_history.db
Established SQLite Database connection: sqlite:///conversation_history.db


I am looking for the color of a laptop

In [43]:
user_id = '0001'

customer_prompt = 'Can you tell about the the features of this laptop?'

chat_with_e_commerce_agent(customer_prompt, user_id)

Established SQLite Database connection: sqlite:///conversation_history.db
Established SQLite Database connection: sqlite:///conversation_history.db


Product Name: Laptop Price: 165.93 Rating: 4.2 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it matches the user's query for a laptop and its features are being        
inquired about.                                                                                                    

Product Name: Laptop Price: 408.08 Rating: 4.8 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it is a laptop option with high ratings and similar features.              

Product Name: Laptop Price: 553.46 Rating: 3.6 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it is another laptop option with similar features.                         

Product Name: Laptop Price: 621.59 Rating: 4.1 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it is a laptop option with similar features and a relatively high rating.  

Product Name: Laptop Price: 710.99 Rating: 3.1 Description: Intel i7 processor; 16GB RAM; 512GB SSD; 15.6-inch FHD 
screen I recommend this product because it is a laptop option with similar features, although with a lower rating.

In [44]:
user_id = '0001'

customer_prompt = 'So what have we discussed so far?'

chat_with_e_commerce_agent(customer_prompt, user_id)

Established SQLite Database connection: sqlite:///conversation_history.db
Established SQLite Database connection: sqlite:///conversation_history.db


You've asked what you've discussed so far, which seems like a generic query. I'm here to help with any questions or
recommendations you might need regarding our products. How can I assist you today?

In [45]:
user_id = '0001'

customer_prompt = 'Who are you and what do you do?'

chat_with_e_commerce_agent(customer_prompt, user_id)

Established SQLite Database connection: sqlite:///conversation_history.db
Established SQLite Database connection: sqlite:///conversation_history.db


You've asked a generic query about who I am and what I do. I'm here to help with any questions or recommendations  
you might need regarding our products. How can I assist you today?